In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import json
import pandas as pd
import os
import sys
sys.path.append('.')
import difflib
import find_diff_writes

In [4]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

In [6]:
def check_fidelity(dirr):
    screenshot_simi = screenshot_similarity[dirr.split('/')[-1]]
    live_html = open(f"{dirr}/live.html").read()
    archive_html = open(f"{dirr}/archive.html").read()
    live_element = json.load(open(f"{dirr}/live_elements.json"))
    archive_element = json.load(open(f"{dirr}/archive_elements.json"))
    my_fidelity = fidelity_check.verify(live_html, live_element, archive_html, archive_element)
    print(screenshot_simi >= 1, my_fidelity)

# check_fidelity('testcases/theftaz.azag.gov_1')

#### Test difflib

In [27]:
e1 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e2 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e1 == e2

/a/b /a/b True


True

## Check archive differences

In [10]:
dirr = 'testcases/www.slideshare.net_1'
dirr = '../execution_match/examples/eta.lbl.gov_2'

live_element = json.load(open(f"{dirr}/live_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element, returnHTML=False)
print(len(live_unique), len(archive_unique))
for i in live_unique:
    print("live:\n", i)
for i in archive_unique:
    print("archive:\n", i)

live_unique number [2, 1, 1, 1, 1, 1, 1, 1]
archive_unique number [2, 1, 1, 1, 1, 1, 7, 1]
8 8
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]', '/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/picture[1]/img[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/a[1]/div[1]/div[2]/svg[1]/path[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/a[1]/div[1]/div[2]/svg[1]/use[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/div[3]/div[1]/div[1]']
live:
 ['/html[1]/body[1

In [99]:
# * Generate testcase
diff = {"live": [], "archive": []}
for lu in live_unique:
    first_line = lu.strip().split('\n')[0]
    diff['live'].append({"root": first_line, "numlines": len(lu.strip().split('\n'))})
for au in archive_unique:
    first_line = au.strip().split('\n')[0]
    diff['archive'].append({"root": first_line, "numlines": len(au.strip().split('\n'))})
print(json.dumps(diff, indent=4))

{
    "live": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--responsive media--image is-b-loaded\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        }
    ],
    "archive": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--loading is-b-loading media--responsive media--image\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<button class=\"ytp-watch-later-button ytp-button ytp-show-watch-later-title\" data-tooltip-opaque=\"true\" title=\"Watch later\" aria-label=\"Watch later\">",
            "numlines": 7
        }
    ]
}


### Check the assumption of archive's writes are always a subset of live

In [14]:
dirs = os.listdir('../record_replay/writes')
for dirr in dirs:
    key_writes = find_diff_writes.locate_key_writes(f'../record_replay/writes/{dirr}')
    live_additional = [len(w['key_related_writes']) for w in key_writes['live']]
    archive_additional = [len(w['key_related_writes']) for w in key_writes['archive']]
    print(dirr)
    print('live:', live_additional)
    print('archive:' archive_additional)
    print('\n')

live_unique number []
archive_unique number [2, 2, 2, 2, 2, 2]
nimhd.nih.gov_1 [] [0, 0, 0, 0, 0, 0]


/home/jingyz/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


live_unique number [2, 1, 1]
archive_unique number [2, 1, 7, 1]
eta.lbl.gov_1 [5, 0, 0] [0, 0, 0, 0]
live_unique number [11, 9, 1, 1]
archive_unique number [1, 1]
www.slideshare.net_1 [22, 22, 1, 1] [0, 0]
live_unique number [11, 7, 1, 1, 1, 1, 47]
archive_unique number [11, 7, 1, 1, 1, 1, 36]
www.nato.int_1 [150, 90, 0, 0, 0, 0, 274] [0, 0, 0, 0, 0, 0, 0]
live_unique number [2, 3, 1, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
archive_unique number [2, 1]
www.sewp.nasa.gov_1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0]
live_unique number [1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 61, 1, 1]
archive_unique number [1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 47, 1, 1]
dra.gov_1 [0, 4, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 72, 6, 6] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
live_unique number [4, 1, 1, 1, 1, 1, 1]
archive_unique number [2, 1, 1, 2, 1, 1, 6, 1, 1, 5, 1, 1, 1, 1, 1, 1

/home/jingyz/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


www.globe.gov_1 [7, 0, 0, 1, 1, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3]
